In [6]:
from PIL import Image, ImageOps
import numpy as np
import time
from copy import deepcopy
from paddleocr import PaddleOCR
from difflib import SequenceMatcher
ocr_model = PaddleOCR(lang='en',use_angle_cls=True,show_log=False)

In [28]:
def unrotate(image,template_size):
    initializers = ['Description']
    
    image = ImageOps.exif_transpose(image)
    img = np.asarray(image)
    data = ocr_model.ocr(img)[0]
    
    start_time = time.time()
    initializing_point = None
    headers = []
    for i in initializers:
        for j in data:
            if i.casefold() in j[1][0].casefold() or SequenceMatcher(None, i.casefold(), j[1][0].casefold()).ratio()>0.7:
                j_ymax = max([k[1] for k in j[0]])
                j_ymin = min([k[1] for k in j[0]])
                initializing_point = j
                
                for k in data:
                    k_ymax = max([n[1] for n in k[0]])
                    k_ymin = min([n[1] for n in k[0]])
                    
                    if not (k_ymax>j_ymax and k_ymin>j_ymax) and not (k_ymax<j_ymin and k_ymin<j_ymin) and not any([n in k[1][0] for n in '1234567890']):
                        headers.append(k)
        if len(headers)>0:
            break
    prev_header_len = len(headers)
    limit = 1
    prev_error = 0
    while(limit!=5):
        all_angles = []
        initializing_xave =  sum([i[0] for i in initializing_point[0]])/4.0
        initializing_yave =  sum([i[1] for i in initializing_point[0]])/4.0
        initializer_mid_point = (initializing_xave,initializing_yave)
        for i in headers:
            i_xave = sum([j[0] for j in i[0]])/4.0
            i_yave = sum([j[1] for j in i[0]])/4.0
            dist = ((i_xave-initializing_xave)**2 + (i_yave-initializing_yave)**2)**0.5
            i_xlen = max([j[0] for j in initializing_point[0]]) - min([j[0] for j in initializing_point[0]])
            if i!=initializing_point and dist<i_xlen*1.5:
                angle = np.arctan2(i_yave-initializing_yave,i_xave-initializing_xave)
                if angle!=0:
                    sign = int(angle/abs(angle))
                else:
                    sign = 1
                dist_to_180 = np.pi - abs(angle)
                dist_to_0 = abs(angle)
                if dist_to_0<dist_to_180:
                    all_angles.append((-1*sign,dist_to_0))
                else:
                    all_angles.append((sign,dist_to_180))
                    
        angle = sorted(all_angles,key=lambda x:x[1])[0]
        data = rotator(data,angle[0]*angle[1],initializer_mid_point)
        headers = header_checker(data,initializers)

        prev_header_len = len(headers)
        prev_error += angle[0]*angle[1]
        limit += 1
    print(np.degrees(prev_error))
    image = image.rotate(-np.degrees(prev_error),center = initializer_mid_point)
    image = image.resize(template_size)
    image.show()
    print("Time Taken: %s seconds" % (time.time() - start_time))
    return data

In [29]:
def rotator(data,angle,initializing_point):
    for i in data:
        points = i[0]
        i_xave = sum([j[0] for j in i[0]])/4.0
        i_yave = sum([j[1] for j in i[0]])/4.0
        rotated_center = rotate_about_point(points,origin = initializing_point,angle = angle)
        i[0] = rotate_about_point(rotated_center,origin=(i_xave,i_yave),angle = -angle)
    return data

In [30]:
def header_checker(data,initializers):
    headers = []
    for i in initializers:
        for j in data:
            if i.casefold() in j[1][0].casefold() or SequenceMatcher(None, i.casefold(), j[1][0].casefold()).ratio()>0.7:
                j_ymax = max([k[1] for k in j[0]])
                j_ymin = min([k[1] for k in j[0]])

                for k in data:
                    k_ymax = max([n[1] for n in k[0]])
                    k_ymin = min([n[1] for n in k[0]])

                    if not (k_ymax>j_ymax and k_ymin>j_ymax) and not (k_ymax<j_ymin and k_ymin<j_ymin) and not any([n in k[1][0] for n in '1234567890']):
                        headers.append(k)
        if len(headers)>0:
            break
    return headers

In [31]:
def rotate_about_point(p, origin=(0, 0), angle=0):
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle),  np.cos(angle)]])
    o = np.atleast_2d(origin)
    p = np.atleast_2d(p)
    return np.squeeze((R @ (p.T-o.T) + o.T).T).tolist()

In [32]:
template_size = (1215, 2689)
img_path = "../data/rotational_test_sample_2.jpg"
# img_path = "../data/rotational_test_sample_2.jpg"
image = Image.open(img_path).convert("RGB")
output = unrotate(image,template_size)

7.291677501969722
Time Taken: 4.644752025604248 seconds


In [33]:
output

[[[[474.36751577002644, 398.2459203900976],
   [1054.3675157700263, 329.2459203900976],
   [1061.3675157700259, 387.24592039009735],
   [481.36751577002633, 455.2459203900975]],
  ('FOODS(SPTELTD', 0.9658881425857544)],
 [[[466.73098278033945, 501.7266429025414],
   [1067.7309827803392, 426.7266429025416],
   [1073.7309827803397, 473.7266429025415],
   [472.73098278033933, 547.7266429025416]],
  ('TEL67785666 FAX67778010', 0.9769943356513977)],
 [[[946.5543781812315, 600.6238737314153],
   [1402.5543781812314, 553.6238737314155],
   [1408.5543781812314, 610.6238737314155],
   [953.5543781812316, 658.6238737314154]],
  ('TAX CASH SALE', 0.9517180919647217)],
 [[[242.02635992207695, 565.737235030847],
   [1290.0263599220768, 443.7372350308472],
   [1298.026359922077, 501.7372350308471],
   [249.02635992207706, 623.7372350308474]],
  ('GST Reg No :MR-8500207-9 Co.Reg No :197801714R', 0.9412440657615662)],
 [[[1121.0145355833274, 660.2148099609211],
   [1406.0145355833272, 630.214809960921